In [3]:
import json
import pandas as pd
import string
import numpy as np
import random
import time

C:\Users\PC\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [81]:
%%writefile test.json
{
    "intents":
    [
        {
            "tag": "Desing",
            "input":["desing", "Desing", "desing", 
                     "design", "desing", "desing",
                    "desing", "desing"],
            "output":[
                "Moda Chique Industria: E-commerce de moda Objetivo: Melhorar a experiencia do usuario e aumentar a taxa de conversao no site. A loja online Moda Chique enfrenta dificuldades com uma taxa de conversao baixa. Muitos usuarios abandonam o site antes de completar a compra. O objetivo do redesign e criar um layout mais intuitivo, atrativo e funcional para melhorar a experiencia do usuario (UX) e incentivar as compras",
                "Empresa: TechWave Industria: Tecnologia e Inovacao Objetivo: Desenvolver uma identidade visual que transmita inovacao, modernidade e confiança. TechWave e uma startup de tecnologia que esta se lancando no mercado e precisa de uma identidade visual forte e consistente para se destacar e atrair investidores, parceiros e clientes",
                "Empresa: GreenBrew Coffee Indústria: Alimentos e Bebidas Objetivo: Desenvolver uma embalagem sustentavel que reflita os valores ecologicos e de alta qualidade do cafe organico da empresa. GreenBrew Coffee esta lancando uma nova linha de cafe organico e precisa de uma embalagem que nao so proteja o produto, mas tambem comunique os valores de sustentabilidade e qualidade premium da marca. A embalagem precisa ser pratica, eco-friendly e atrativa para os consumidores."
            ]
        },
        {
            "tag":"dados",
            "input":["ciencia de dados"," dados","dados", "dados", "ciencia de dados", "dados",
                     "ciencia de dados"],
            "output":[
                "Classificacao de tumor cerebral: O tumor cerebral e considerado uma das doencas mais agressivas, entre criancas  e adultos. Os tumores cerebrais sao responsavel por 85 a 90 por cento de todos os tumores primarios do Sistema Nervoso Central(SNC) A aplicacao de tecnicas de classificacao automatizada usando Machine Learning e Inteligencia Artificial tem consistentemente mostrado maior precisao do que a classificacao manual assim propomos a criacao de um algoritmo de aprendizado profundo utlizando Convolution Neural Network(CNN) link: https://www.kaggle.com/datasets/gonzajl/tumores-cerebrales-mri-dataset",   
                "Previsao de fraude de cartao de credito: Esse conjunto de dados oferece  uma variedade de atributos valiosos para uma analise abrangente. Ele contem 555.719 instancia e 22 atributos, uma mistura de tipos de dados categoricos e numericos link: https://www.kaggle.com/datasets/kelvimimperial/deteco-de-fraudes-em-cartes-de-credito",
                "Conjuto de dados de medicamentos desde o uso, efeitos colaterais e avaliacao: Este conjunto de dados, fornece informacoes abrangentes sobre varios medicamentos, incluindo sua composicao, usos, efeitos detalhe do fabricante, avaliacoes de usuarios, etc. link: https://www.kaggle.com/datasets/prathamtripathi/drug-classification",
                "Previsao de sobrevivencia de pacientes com insuficiencia cardiaca: Esse conjunto de dados contem os prontuarios de 299 pacientes que apresentaram insuficiencia cardiaca, coletados durante o pariodo de seguimento, onde cada perfil de paciente apresenta 13 caracteristicas clinicas",
                "PNL no Facebook: A onipresenca dos smartphones permite que as pessoas anunciem uma emergencia que estao observando em tempo real. Por causa disso, mais agencias estao interessados em monitorar programaticamente o Facebook(ou seja, organizacoes de ajuda em desastres e agencia de noticias). Mas, nem sempre esta claro se as palavras de uma pessoa estao realmente anunciando um desastre. Link: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis"
                     ]
        }
    ]
    
    
}

Overwriting test.json


In [82]:
X = []
y = []
resposta = {}

In [83]:
with open("test.json") as test:
    data = json.load(test)

In [84]:
data

{'intents': [{'tag': 'Desing',
   'input': ['desing',
    'Desing',
    'desing',
    'design',
    'desing',
    'desing',
    'desing',
    'desing'],
   'output': ['Moda Chique Industria: E-commerce de moda Objetivo: Melhorar a experiencia do usuario e aumentar a taxa de conversao no site. A loja online Moda Chique enfrenta dificuldades com uma taxa de conversao baixa. Muitos usuarios abandonam o site antes de completar a compra. O objetivo do redesign e criar um layout mais intuitivo, atrativo e funcional para melhorar a experiencia do usuario (UX) e incentivar as compras',
    'Empresa: TechWave Industria: Tecnologia e Inovacao Objetivo: Desenvolver uma identidade visual que transmita inovacao, modernidade e confianÃ§a. TechWave e uma startup de tecnologia que esta se lancando no mercado e precisa de uma identidade visual forte e consistente para se destacar e atrair investidores, parceiros e clientes',
    'Empresa: GreenBrew Coffee IndÃºstria: Alimentos e Bebidas Objetivo: Desen

In [85]:
for intent in data['intents']:
    resposta[intent['tag']] = intent['tag']
    for row in intent['input']:
        X.append(row)
        y.append(intent['tag'])

In [86]:
df = pd.DataFrame(data={'X':X, 'y':y})

In [87]:
df

,X,y
0,desing,Desing
1,Desing,Desing
2,desing,Desing
3,design,Desing
4,desing,Desing
5,desing,Desing
6,desing,Desing
7,desing,Desing
8,ciencia de dados,dados
9,dados,dados


In [88]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [89]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['X'])
train = tokenizer.texts_to_sequences(df['X'])
Xtrain = pad_sequences(train)

In [90]:
from sklearn.preprocessing import LabelEncoder

In [91]:
le = LabelEncoder()

In [92]:
ytrain = le.fit_transform(df['y'])

In [93]:
input_shape = Xtrain.shape[1]

In [94]:
input_shape

3

In [95]:
vocabulary = len(tokenizer.word_index)
output_length = le.classes_.shape[0]

In [96]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Flatten, Dense, Embedding, Input

In [97]:
i = Input(shape=(input_shape, ))
x = Embedding(vocabulary + 1, 10)(i)
x = LSTM(10, return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length, activation='softmax')(x)
model = Model(i, x)

In [98]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [99]:
model.fit(Xtrain, ytrain, epochs=54, batch_size=2)

Epoch 1/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.5537 - loss: 0.6923
Epoch 2/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7108 - loss: 0.6890 
Epoch 3/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6720 - loss: 0.6866 
Epoch 4/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8016 - loss: 0.6773 
Epoch 5/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8530 - loss: 0.6690 
Epoch 6/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8247 - loss: 0.6651 
Epoch 7/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5470 - loss: 0.6777     
Epoch 8/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8016 - loss: 0.6469 
Epoch 9/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7849 - loss: 0.6444
Epoch 10/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6396 - loss: 0.6448 
Epoch 11/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8173 - loss: 0.6008 
Epoch 12/54
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7183 - loss: 0.6008

In [100]:
while True:
    text = []
    predict = input('Área de interesse: ')
    time.sleep(0.6)
    if predict == 'sair':
        break
    predict = [letters.lower() for letters in predict if letters not in string.punctuation]
    predict = ''.join(predict)
    text.append(predict)

    predict = tokenizer.texts_to_sequences(text)
    predict = np.array(predict).reshape(-1)
    predict = pad_sequences([predict], input_shape)

    output = model.predict(predict)
    output = output.argmax()

    resp = le.inverse_transform([output])[0]
    if resp == 'Desing':
        saida =  data['intents']
        aleatorio = random.randint(0, 3)
        print(saida[0]['output'][aleatorio])
    elif resp == 'dados':
        saida =  data['intents']
        aleatorio = random.randint(0, 2)
        print(saida[1]['output'][aleatorio])

Área de interesse: design
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step
Empresa: TechWave Industria: Tecnologia e Inovacao Objetivo: Desenvolver uma identidade visual que transmita inovacao, modernidade e confianÃ§a. TechWave e uma startup de tecnologia que esta se lancando no mercado e precisa de uma identidade visual forte e consistente para se destacar e atrair investidores, parceiros e clientes
Área de interesse: dados
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Classificacao de tumor cerebral: O tumor cerebral e considerado uma das doencas mais agressivas, entre criancas  e adultos. Os tumores cerebrais sao responsavel por 85 a 90 por cento de todos os tumores primarios do Sistema Nervoso Central(SNC) A aplicacao de tecnicas de classificacao automatizada usando Machine Learning e Inteligencia Artificial tem consistentemente mostrado maior precisao do que a classificacao manual assim propomos a criacao de um algoritmo de aprendizado profundo utlizando Convolution Neural Network(CNN) link: https